In [1]:
from utils import NerDataset

dev_dataset = NerDataset('./processed/processed_dev_bio.txt')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
len(dev_dataset.sents)

909

In [4]:
train_dataset = NerDataset('./processed/processed_training_bio.txt')
len(train_dataset)

6306

In [5]:
test_dataset[0]

('[CLS] ， 患 者 3 月 前 因 “ 直 肠 癌 ” 于 在 我 院 于 全 麻 上 行 直 肠 癌 根 治 术 （ D I X O N 术 ） ， 手 术 过 程 顺 利 ， 术 后 给 予 抗 感 染 及 营 养 支 持 治 疗 ， 患 者 恢 复 好 ， 切 口 愈 合 良 好 。 ， 术 后 病 理 示 ： 直 肠 腺 癌 （ 中 低 度 分 化 ） ， 浸 润 溃 疡 型 ， 面 积 3 . 5 * 2 C M ， 侵 达 外 膜 。 双 端 切 线 另 送 “ 近 端 ” 、 “ 远 端 ” 及 环 周 底 部 切 除 面 未 查 见 癌 。 肠 壁 一 站 （ 1 0 个 ） 、 中 间 组 （ 8 个 ） 淋 巴 结 未 查 见 癌 。 ， 免 疫 组 化 染 色 示 ： E R C C 1 弥 漫 （ + ） 、 T S 少 部 分 弱 （ + ） 、 S Y N （ - ） 、 C G A （ - ） 。 术 后 查 无 化 疗 禁 忌 后 给 予 3 周 期 化 疗 ， ， 方 案 为 ： 奥 沙 利 铂 1 5 0 M G O D 1 ， 亚 叶 酸 钙 0 . 3 G + 替 加 氟',
 [101,
  8024,
  2642,
  5442,
  124,
  3299,
  1184,
  1728,
  100,
  4684,
  5499,
  4617,
  100,
  754,
  1762,
  2769,
  7368,
  754,
  1059,
  7937,
  677,
  6121,
  4684,
  5499,
  4617,
  3418,
  3780,
  3318,
  8020,
  146,
  151,
  166,
  157,
  156,
  3318,
  8021,
  8024,
  2797,
  3318,
  6814,
  4923,
  7556,
  1164,
  8024,
  3318,
  1400,
  5314,
  750,
  2834,
  2697,
  3381,
  1350,
  5852,
  1075,
  3118,
  2898,
  3780,
  4545,
  8024,
  2642,
  5442,
  261

In [9]:
train_dataset[1]

('[CLS] ， 患 者 因 罹 患 “ 胃 癌 ” 于 2 0 1 3 - 1 0 - 2 9 在 我 院 予 行 全 麻 上 胃 癌 根 治 术 ， ， 术 中 见 ： 腹 腔 内 腹 水 ， 腹 膜 无 转 移 ， 肝 脏 未 触 及 明 显 转 移 性 灶 ， 肿 瘤 位 于 胃 体 、 胃 底 部 ， 小 弯 侧 偏 后 壁 ， 约 5 * 4 * 2 C M 大 小 ， 肿 瘤 已 侵 达 浆 膜 外 ， 第 1 、 3 组 淋 巴 结 肿 大 ， 肿 瘤 尚 能 活 动 ， 经 探 查 决 定 行 全 胃 切 除 ， 空 肠 J 字 代 胃 术 。 手 术 顺 利 ， 术 后 积 极 予 相 关 对 症 支 持 治 疗 ； ， 后 病 理 示 ： 胃 底 、 体 小 弯 侧 低 分 化 腺 癌 ， 部 分 为 印 戒 细 胞 癌 图 像 ， 蕈 伞 型 ， 面 积 5 . 2 * 3 . 5 C M ， 局 部 侵 达 粘 膜 上 层 ， 并 于 少 数 腺 管 内 查 见 癌 栓 。 双 端 切 线 及 另 送 “ 近 端 切 线 ” 未 查 见 癌 。 呈 三 组 （ 5 / 1 3 个 ） 淋 巴',
 [101,
  8024,
  2642,
  5442,
  1728,
  5395,
  2642,
  100,
  5517,
  4617,
  100,
  754,
  123,
  121,
  122,
  124,
  118,
  122,
  121,
  118,
  123,
  130,
  1762,
  2769,
  7368,
  750,
  6121,
  1059,
  7937,
  677,
  5517,
  4617,
  3418,
  3780,
  3318,
  8024,
  8024,
  3318,
  704,
  6224,
  8038,
  5592,
  5579,
  1079,
  5592,
  3717,
  8024,
  5592,
  5606,
  3187,
  6760,
  4919,
  8024,
  5498,
  5552,
  3313,
  6239,
  1350,
  3209,
  3227,
  6760,
  4919,
 

### 分段处理，使长度低于256

In [37]:
MAX_LEN = 256 - 2
sents, tags_li = [], [] # list of lists
for entry in entries:
    words = [line.split()[0] for line in entry.splitlines()]
    tags = ([line.split()[-1] for line in entry.splitlines()])
#     sents.append(["[CLS]"] + words + ["[SEP]"])  # 每个句子前后加['CLS']和['SEP']
#     tags_li.append(["<PAD>"] + tags + ["<PAD>"])
    if len(words) > MAX_LEN:
        # 先对句号分段
        word, tag = [], []
        for char, t in zip(words, tags):
            
            if char != '。':
                word.append(char)
                tag.append(t)
            else:
#                 if len(word) > MAX_LEN:
#                     word = word[:MAX_LEN]
#                     tag = tag[:MAX_LEN]
                sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
                tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
                word, tag = [], []            
        # 最后的
        if len(word):
            sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
            tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
            word, tag = [], []
    else:
        sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
        tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])

In [38]:
len(sents)

6306

In [39]:
len(tags_li)

6306

In [36]:
sents[0], tags_li[0]

(['，',
  '患',
  '者',
  '3',
  '月',
  '前',
  '因',
  '“',
  '直',
  '肠',
  '癌',
  '”',
  '于',
  '在',
  '我',
  '院',
  '于',
  '全',
  '麻',
  '上',
  '行',
  '直',
  '肠',
  '癌',
  '根',
  '治',
  '术',
  '（',
  'D',
  'I',
  'X',
  'O',
  'N',
  '术',
  '）',
  '，',
  '手',
  '术',
  '过',
  '程',
  '顺',
  '利',
  '，',
  '术',
  '后',
  '给',
  '予',
  '抗',
  '感',
  '染',
  '及',
  '营',
  '养',
  '支',
  '持',
  '治',
  '疗',
  '，',
  '患',
  '者',
  '恢',
  '复',
  '好',
  '，',
  '切',
  '口',
  '愈',
  '合',
  '良',
  '好',
  '。',
  '，',
  '术',
  '后',
  '病',
  '理',
  '示',
  '：',
  '直',
  '肠',
  '腺',
  '癌',
  '（',
  '中',
  '低',
  '度',
  '分',
  '化',
  '）',
  '，',
  '浸',
  '润',
  '溃',
  '疡',
  '型',
  '，',
  '面',
  '积',
  '3',
  '.',
  '5',
  '*',
  '2',
  'C',
  'M',
  '，',
  '侵',
  '达',
  '外',
  '膜',
  '。',
  '双',
  '端',
  '切',
  '线',
  '另',
  '送',
  '“',
  '近',
  '端',
  '”',
  '、',
  '“',
  '远',
  '端',
  '”',
  '及',
  '环',
  '周',
  '底',
  '部',
  '切',
  '除',
  '面',
  '未',
  '查',
  '见',
  '癌',
  '。',
  '肠',
  '壁',
  '一',
  '站',

In [2]:
from pytorch_pretrained_bert import BertModel
bert = BertModel.from_pretrained('bert-base-chinese')

In [4]:
x = dev_dataset[1]

In [9]:
import torch
encode_layers, _ = bert(torch.tensor(x[1]).unsqueeze(0))
enc = encode_layers[-1]
enc

tensor([[[-0.4081,  0.6838, -0.1342,  ..., -0.6718, -0.5864,  0.2491],
         [-0.1821,  0.4146,  0.9377,  ..., -1.1175, -0.4919,  0.0507],
         [-0.2332,  0.3116,  0.1658,  ...,  0.4438,  0.3029, -0.0686],
         ...,
         [-0.0225, -0.1823, -0.4035,  ...,  0.8034, -0.0065, -0.0556],
         [-0.5497,  0.5112,  1.1299,  ...,  0.3418, -0.0035, -0.6358],
         [-0.2669, -0.1594, -0.0355,  ...,  0.3744, -0.3602, -0.0103]]],
       grad_fn=<AddBackward0>)

In [10]:
enc.size()

torch.Size([1, 59, 768])

In [11]:
len(x[1])

59

In [14]:
enc = enc.view(len(x[1]), 1, -1)
enc.size()

torch.Size([59, 1, 768])

In [11]:
from utils import NerDataset
dev_dataset = NerDataset('./processed/processed_dev_bio.txt')

In [12]:
dev_dataset.sents[0]

['[CLS]',
 '，',
 '患',
 '者',
 '2',
 '0',
 '1',
 '1',
 '年',
 '5',
 '月',
 '1',
 '日',
 '因',
 '尿',
 '频',
 '尿',
 '急',
 '在',
 '外',
 '院',
 '检',
 '查',
 '，',
 '行',
 'B',
 '超',
 '及',
 'C',
 'T',
 '检',
 '查',
 '提',
 '示',
 '盆',
 '腔',
 '包',
 '块',
 '，',
 '建',
 '议',
 '进',
 '一',
 '步',
 '检',
 '查',
 '[SEP]']

In [13]:
dev_dataset.tags_li[0]

['<PAD>',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-INF',
 'I-INF',
 'O',
 'B-INF',
 'I-INF',
 'O',
 'O',
 'O',
 'O',
 'B-PAT',
 'I-PAT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '<PAD>']

In [14]:
for sent, tag in zip(dev_dataset.sents, dev_dataset.tags_li):
    assert len(sent) == len(tag), f"{len(sent)}, {len(tag)}"

In [7]:
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('/root/workspace/qa_project/chinese_L-12_H-768_A-12/')

In [8]:
VOCAB = ('<PAD>', 'O', 'B-INF', 'I-INF', 'B-PAT', 'I-PAT', 'B-OPS', 
        'I-OPS', 'B-DSE', 'I-DSE', 'B-DRG', 'I-DRG', 'B-LAB', 'I-LAB')
tag2idx = {tag: idx for idx, tag in enumerate(VOCAB)}

In [15]:
idx = 0
for words, tags in zip(dev_dataset.sents, dev_dataset.tags_li):
    x, y = [], []
    is_heads = []
    for w, t in zip(words, tags):
        tokens = tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
        xx = tokenizer.convert_tokens_to_ids(tokens)
        
        is_head = [1] + [0]*(len(tokens) - 1)
        t = [t] + ["<PAD>"] * (len(tokens) - 1)
        yy = [tag2idx[each] for each in t]  # (T,)

        x.extend(xx)
        is_heads.extend(is_head)
        y.extend(yy)
    assert len(x)==len(y)==len(is_heads), f"{idx}"
    idx += 1


AssertionError: 366

In [16]:
dev_dataset.sents[366]

['[CLS]',
 '术',
 '后',
 '行',
 '2',
 '周',
 '期',
 '辅',
 '助',
 '化',
 '疗',
 '，',
 '方',
 '案',
 'M',
 'F',
 'O',
 'L',
 'F',
 'O',
 'X',
 '7',
 '(',
 'O',
 'X',
 'A',
 '1',
 '0',
 '0',
 'M',
 'G',
 '/',
 'M',
 '\ue236',
 ',',
 'C',
 'F',
 '4',
 '0',
 '0',
 'M',
 'G',
 '/',
 'M',
 '\ue236',
 ',',
 '5',
 'F',
 'U',
 '2',
 '4',
 '0',
 '0',
 'M',
 'G',
 '/',
 'M',
 '\ue236',
 ')',
 '，',
 '过',
 '程',
 '顺',
 '利',
 '[SEP]']

### CRF

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

In [16]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

In [17]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [18]:
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [19]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

In [20]:
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_to_ix

{'the': 0,
 'wall': 1,
 'street': 2,
 'journal': 3,
 'reported': 4,
 'today': 5,
 'that': 6,
 'apple': 7,
 'corporation': 8,
 'made': 9,
 'money': 10,
 'georgia': 11,
 'tech': 12,
 'is': 13,
 'a': 14,
 'university': 15,
 'in': 16}

In [21]:
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

In [23]:
precheck_sent = prepare_sequence(training_data[1][0], word_to_ix)
precheck_sent

tensor([11, 12, 13, 14, 15, 16, 11])

In [24]:
precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]],
                            dtype=torch.long)
precheck_tags

tensor([0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])

In [25]:
### 过模型
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [26]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)

In [27]:
lstm_feates = model._get_lstm_features(precheck_sent)
lstm_feates

tensor([[-0.0940,  0.2521, -0.1978,  0.7061, -0.1087],
        [ 0.0075, -0.0180, -0.3852,  0.5408, -0.0325],
        [-0.2446,  0.3031, -0.3457,  0.4786, -0.3029],
        [-0.2421,  0.1327, -0.3639,  0.5181, -0.2309],
        [-0.4148,  0.1546, -0.4694,  0.4675, -0.0463],
        [-0.2203,  0.2797, -0.2896,  0.4339, -0.6936],
        [-0.1195,  0.1382, -0.2920,  0.4968, -0.5249]],
       grad_fn=<AddmmBackward>)

In [28]:
lstm_feates.size()  # 5是标签长度， 7是文本长度

torch.Size([7, 5])

In [29]:
### 前向算法
init_alphas = torch.full((1, model.tagset_size), -10000.)
init_alphas

tensor([[-10000., -10000., -10000., -10000., -10000.]])

In [30]:
init_alphas[0][model.tag_to_ix[START_TAG]] = 0.
init_alphas

tensor([[-10000., -10000., -10000.,      0., -10000.]])

In [31]:
forward_var = init_alphas

In [40]:
for feat in lstm_feates:
    alphas_t = []
    for next_tag in range(model.tagset_size):
        emit_score = feat[next_tag].view(1, 
                                -1).expand(1, model.tagset_size)
        #print(emit_score)
        #break
        trans_score = model.transitions[next_tag].view(1, -1)
        # print(trans_score)
        # break
        next_tag_var = forward_var + trans_score + emit_score
        
        alphas_t.append(log_sum_exp(next_tag_var).view(1))
    # 改变
    forward_var = torch.cat(alphas_t).view(1, -1)
alphas_t

[tensor([4.3112], grad_fn=<ViewBackward>),
 tensor([5.4447], grad_fn=<ViewBackward>),
 tensor([5.2821], grad_fn=<ViewBackward>),
 tensor([-9993.8789], grad_fn=<ViewBackward>),
 tensor([4.6432], grad_fn=<ViewBackward>)]

In [34]:
terminal_var = torch.cat(alphas_t).view(1, -1)

tensor([-0.1195,  0.1382, -0.2920,  0.4968, -0.5249], grad_fn=<SelectBackward>)

In [38]:
model.transitions

Parameter containing:
tensor([[-1.1811e-01, -1.4420e+00, -1.1108e+00, -1.1187e+00, -1.0000e+04],
        [-4.9566e-01, -1.9700e-01, -3.3396e-02,  1.4273e+00, -1.0000e+04],
        [-7.5307e-01, -4.3190e-01,  6.6930e-01,  6.5051e-01, -1.0000e+04],
        [-1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04],
        [ 1.8568e-01, -2.7636e-01, -5.9385e-01, -3.0606e-01, -1.0000e+04]],
       requires_grad=True)

In [41]:
model.word_embeds

Embedding(17, 5)

In [44]:
embeds = model.word_embeds(precheck_sent).view(len(precheck_sent), 1, -1)
embeds.size()

torch.Size([7, 1, 5])

In [43]:
sentence

['georgia', 'tech', 'is', 'a', 'university', 'in', 'georgia']

In [46]:
hidden = model.init_hidden()
hidden

(tensor([[[-0.1786, -0.3710]],
 
         [[-0.5775,  0.3241]]]), tensor([[[ 0.6319,  0.1522]],
 
         [[-1.2962,  0.9093]]]))

In [47]:
lstm_output, hidden = model.lstm(embeds, hidden)

In [48]:
lstm_output.size()

torch.Size([7, 1, 4])

In [60]:
lstm_out = lstm_output.view(len(precheck_sent), model.hidden_dim)
lstm_out.size()

torch.Size([7, 4])

In [61]:
lstm_feates = model.hidden2tag(lstm_out)
lstm_feates.size()

torch.Size([7, 5])

In [49]:
from pytorch_pretrained_bert import BertModel
bert = BertModel.from_pretrained('bert-base-chinese')

In [50]:
precheck_sent

tensor([11, 12, 13, 14, 15, 16, 11])

In [52]:
from utils import NerDataset
training_data = NerDataset('./processed/processed_training_bio.txt')
training_data[0]

('[CLS] ， 患 者 3 月 前 因 “ 直 肠 癌 ” 于 在 我 院 于 全 麻 上 行 直 肠 癌 根 治 术 （ D I X O N 术 ） ， 手 术 过 程 顺 利 ， 术 后 给 予 抗 感 染 及 营 养 支 持 治 疗 ， 患 者 恢 复 好 ， 切 口 愈 合 良 好 [SEP]',
 [101,
  8024,
  2642,
  5442,
  124,
  3299,
  1184,
  1728,
  100,
  4684,
  5499,
  4617,
  100,
  754,
  1762,
  2769,
  7368,
  754,
  1059,
  7937,
  677,
  6121,
  4684,
  5499,
  4617,
  3418,
  3780,
  3318,
  8020,
  146,
  151,
  166,
  157,
  156,
  3318,
  8021,
  8024,
  2797,
  3318,
  6814,
  4923,
  7556,
  1164,
  8024,
  3318,
  1400,
  5314,
  750,
  2834,
  2697,
  3381,
  1350,
  5852,
  1075,
  3118,
  2898,
  3780,
  4545,
  8024,
  2642,
  5442,
  2612,
  1908,
  1962,
  8024,
  1147,
  1366,
  2689,
  1394,
  5679,
  1962,
  102],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [54]:
for batch in train_iter:
    data = batch
    break
data

(['[CLS] 患 者 输 注 氨 磷 汀 恶 心 ， 停 输 [SEP]',
  '[CLS] 自 第 下 次 化 疗 结 束 出 院 以 来 ， 患 者 未 诉 明 显 不 适 ， 一 般 情 况 保 持 良 好 ； 无 发 热 ， 无 恶 心 、 呕 吐 ， 无 反 酸 、 嗳 气 ， 无 明 显 进 食 不 适 ， 未 现 明 显 腹 痛 、 腹 胀 [SEP]',
  '[CLS] 此 次 为 化 疗 再 次 入 院 [SEP]',
  '[CLS] 下 次 出 院 以 来 精 神 、 睡 眠 、 饮 食 可 ， 无 腹 痛 、 腹 胀 、 发 热 ， 大 小 便 正 常 ， 体 重 较 前 无 明 显 变 化 [SEP]',
  '[CLS] B-PAT 一 站 淋 巴 结 （ 8 个 ） 未 查 见 癌 [SEP]',
  '[CLS] 肝 内 多 发 结 节 ， 考 虑 转 移 瘤 ， 部 分 较 前 新 发 ， 肝 S 4 病 灶 较 前 增 大 [SEP]',
  '[CLS] 病 理 诊 断 O （ 胃 窦 ） 慢 性 萎 缩 性 胃 炎 （ 中 度 ） ， 局 部 腺 体 肠 下 皮 化 生 [SEP]',
  '[CLS] ， 缘 于 入 院 前 4 月 余 于 我 院 诊 断 为 胃 窦 癌 ， 于 2 0 1 5 年 0 8 月 2 5 日 在 全 麻 上 行 “ 腹 腔 镜 辅 助 上 根 治 性 远 端 胃 大 部 切 除 + 残 胃 空 肠 R O U X - Y 吻 合 术 （ D 2 ） + 阑 尾 切 除 术 ” ， 手 术 顺 利 [SEP]'],
 tensor([[ 101, 2642, 5442, 6783, 3800, 3710, 4840, 3722, 2626, 2552, 8024,  977,
          6783,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,

In [55]:
words, x, is_heads, tags, y, seqlens = data
x.size()

torch.Size([8, 85])

In [58]:
encoded_layers, _ = bert(x)
enc = encoded_layers[-1]
enc.size()

torch.Size([8, 85, 768])

In [63]:
fc = nn.Linear(768, 16)
lstm_feats = fc(enc)
lstm_feats.size()

torch.Size([8, 85, 16])

In [64]:
lstm_out

tensor([[ 0.2317, -0.3432, -0.2300,  0.2327],
        [ 0.0843, -0.6124,  0.0237,  0.0466],
        [ 0.0817, -0.0319, -0.0956, -0.0518],
        [-0.0460, -0.2713,  0.0071,  0.1377],
        [ 0.0196, -0.1559,  0.4119,  0.0321],
        [-0.2767,  0.0784, -0.5070,  0.1274],
        [-0.2274, -0.2640, -0.4283,  0.2956]], grad_fn=<ViewBackward>)

In [ ]:
forward_score = model._score_sentence(lstm_feates, )

In [66]:
precheck_tags

tensor([0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])

In [67]:
gold_score = model._score_sentence(lstm_feates, precheck_tags)
gold_score

tensor([-2.3651], grad_fn=<AddBackward0>)

In [68]:
score = torch.zeros(1)
score

tensor([0.])

In [69]:
tags = torch.cat([torch.tensor([model.tag_to_ix[START_TAG]],
                              dtype=torch.long), precheck_tags])
tags

tensor([3, 0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])

In [71]:
for i, feat in enumerate(lstm_feates):
    score = score + model.transitions[tags[i+1], tags[i]] + feat[tags[i+1]]
score

tensor([-1.7712], grad_fn=<AddBackward0>)

In [72]:
score = score + model.transitions[model.tag_to_ix[STOP_TAG], tags[-1]]
score

tensor([-2.3651], grad_fn=<AddBackward0>)

In [76]:
from crf import Bert_BiLSTM_CRF# , tag2idx
bert_crf = Bert_BiLSTM_CRF(tag2idx)

AttributeError: cannot assign module before Module.__init__() call

In [75]:
BERT_VOCAB = ('<PAD>', '[CLS]', '[SEP]', 'O', 'B-INF', 'I-INF', 'B-PAT', 'I-PAT', 'B-OPS', 
        'I-OPS', 'B-DSE', 'I-DSE', 'B-DRG', 'I-DRG', 'B-LAB', 'I-LAB')
tag2idx = {tag: idx for idx, tag in enumerate(BERT_VOCAB)}

In [7]:
# -*- encoding: utf-8 -*-
'''
@File    :   crf.py
@Time    :   2019/11/23 17:35:36
@Author  :   Cao Shuai
@Version :   1.0
@Contact :   caoshuai@stu.scu.edu.cn
@License :   (C)Copyright 2018-2019, MILAB_SCU
@Desc    :   None
'''
 
import torch
import torch.nn as nn
from pytorch_pretrained_bert import BertModel

BERT_VOCAB = ('<PAD>', '[CLS]', '[SEP]', 'O', 'B-INF', 'I-INF', 'B-PAT', 'I-PAT', 'B-OPS', 
        'I-OPS', 'B-DSE', 'I-DSE', 'B-DRG', 'I-DRG', 'B-LAB', 'I-LAB')
tag2idx = {tag: idx for idx, tag in enumerate(BERT_VOCAB)}
idx2tag = {idx: tag for idx, tag in enumerate(BERT_VOCAB)}

class Bert_BiLSTM_CRF(nn.Module):
    def __init__(self, tag_to_ix, hidden_dim=768):
        super(Bert_BiLSTM_CRF, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.tag2ix = tag_to_ix
        self.target_size = len(tag_to_ix)
        # self.hidden = self.init_hidden()
        self.lstm = nn.LSTM(bidirectional=True, num_layers=2, input_size=768, hidden_size=hidden_dim//2, batch_first=True)
        self.transtions = nn.Parameter(torch.randn(
            self.target_size, self.target_size
        ))
        self.fc = nn.Linear(hidden_dim, self.target_size)
        
        self.bert.eval()  # 知用来取bert embedding
        self.transitions.data[tag_to_ix['[CLS]'], :] = -10000
        self.transitions.data[:, tag_to_ix['[SEP]']] = -10000


    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))


    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix['[CLS]']] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix['[SEP]']]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix['[CLS]']], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix['[SEP]'], tags[-1]]
        return score

    def _bert_enc(self, x):
        """
        x: [batchsize, sent_len]
        enc: [batch_size, sent_len, 768]
        """
        encoded_layer, _  = self.bert(x)
        enc = encoded_layer[-1]
        return enc

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score


    def _get_lstm_features(self, sentence):
        """sentence is the ids"""
        self.hidden = self.init_hidden()
        embeds = self._bert_enc(sentence)  # [8, 75, 768]
        # 过lstm
        enc, _ = self.lstm(embeds)
        lstm_feats = self.fc(enc)
        return lstm_feats  # [8, 75, 16]

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [8]:
bert_crf = Bert_BiLSTM_CRF(tag2idx)

Model name 'bert-base-chinese' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz' was a path or url but couldn't find any file associated to this path or url.


AttributeError: 'NoneType' object has no attribute 'eval'

In [9]:
bert = BertModel.from_pretrained('bert-base-chinese')

  0%|          | 52224/382072689 [00:22<348:13:09, 304.74B/s]

KeyboardInterrupt: 